# Proxy for ACL and parceled control

In [1]:
from dataclay import Client

/home/alex/BSC/training-dataclay-2025/venv/lib/python3.10/site-packages/pydantic_settings/sources.py:659: UserWarning: directory "/run/secrets" does not exist
  warnings.warn(f'directory "{path}" does not exist')


## Connect to a proxy

We started the proxy with middleware in port 8677 (see `docker-compose.yml`).

We connect the client to that. Given that the port is not the standard one (the standard was used in prior exercises), we need to explicitly put it there. Also the username.

In [2]:
client = Client(proxy_host="127.0.0.1", proxy_port=8677, username="john")
client.start()

INFO:dataclay.client.api:Starting client runtime
INFO:dataclay.client.api:Creating event loop in new thread
INFO:dataclay.client.api:Starting client runtime coroutine in event loop
INFO:dataclay.client.api:Client runtime started


Starting event loop in new thread


INFO:dataclay.backend.client:SSL not configured


For this simple example, we will be using the `Person` class from the `modeltest.family` module:

In [3]:
from dataclay.contrib.modeltest.family import Person

p = Person("Alice", 42)
p.make_persistent("alice")

AlreadyExistError: Unexpected <class 'grpc._channel._InactiveRpcError'>: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "admin/alice"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:172.23.0.4:16587 {grpc_message:"admin/alice", grpc_status:13, created_time:"2025-02-06T15:00:03.519676224+00:00"}"
> already exists

Well, actually today's Alice's birthday; let's add a year to her age

In [4]:
p.add_year()

Oh shoot! John can only read, not call methods :(

We can in fact read the attributes:

In [5]:
print(p.age)
print(p.name)

43
Alice


And we cannot, in fact, write to them:

In [6]:
p.name = "Alice Carroll"

## Add a new Middleware for user **james**

Create a new middleware class in `middleware.py`.

- It should apply to the `james` user only
- User `james` should be able to *read* and *write* on the `age` attribute
  - You can check the name of the attribute on the `*ObjectAttribute` through the `request.attribute`.
- User `james` should be able to call the `add_year` method
  - You can check the method name on the `CallActiveMethod` through the `request.method_name`.
 
Once you have finished the middleware.py, restart the `proxy_exercise` docker (`docker compose restart proxy_exercise`).

In [7]:
client = Client(proxy_host="127.0.0.1", proxy_port=8677, username="james")
client.start()

INFO:dataclay.client.api:Starting client runtime
INFO:dataclay.client.api:Using existing event loop
INFO:dataclay.client.api:Starting client runtime coroutine in event loop
INFO:dataclay.client.api:Client runtime started
INFO:dataclay.backend.client:SSL not configured


Retrieving the user and checking its age should be ok:

In [8]:
p = Person.get_by_alias("alice")
p.age

44

This should fail:

In [9]:
p.name

DataClayException: James can only access the `age` attribute

Writing to the attribute or calling to the `add_year` should also be ok

In [10]:
print("Age before adding a year:", p.age)
p.add_year()
print("Age after adding a year:", p.age)

Age before adding a year: 44
Age after adding a year: 45
